### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{},"{'exp_name': 'bc', 'overrides': [], 'config_di...",3fu0komu
1,"{'_timestamp': 1666583734, 'pos_neg_diff': 0.6...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",2s0eqs8w
2,"{'loss': 1.343022703409195, 'idm_loss': 2.9079...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",1gay5kzk
3,"{'epoch': 2, 'pos_sim': 0.33899286955595015, '...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",3iyjeqa6
4,"{'max_grad_norm': 0.2491480689495802, 'loss': ...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",31c7ofut
...,...,...,...
752,"{'_runtime': 20190, '_timestamp': 1659282969, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
753,"{'_timestamp': 1659280381, 'negative_sim': 0.5...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
754,"{'_timestamp': 1659280529, 'negative_sim': 0.1...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
755,"{'pos_neg_diff': 0.810436166524887, 'positive_...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline_10'
exp_name = 'bc'
model_path = '0/100/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_46894/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
9,"{'act_f1': 0.36722800543241735, 'act_acc': 0.4...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",2dgjs3c6,UpNDown
10,"{'act_f1': 0.36934895892808706, 'act_acc': 0.4...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",1hndgvuz,Seaquest
23,"{'best_metric_val': 0.4596222254457859, 'loss'...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",1h0t64dp,Qbert
24,"{'loss': 1.2039528935774226, '_step': 234400, ...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",246wv6gf,Pong
25,"{'loss': 1.6509162017277308, 'act_f1': 0.38444...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",35k3n89v,RoadRunner
26,"{'epoch': 100, 'reward_f1': 0.2799999999999999...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",yybwqakt,PrivateEye
74,"{'epoch': 100, 'act_f1': 0.2589033215565031, '...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",31ktobwm,Krull
75,"{'mean_grad_norm': 0.16100724482147916, 'loss'...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",z7un794r,Kangaroo
76,"{'best_metric_val': 0.4094546521872494, 'act_f...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",1vg8yjnp,KungFuMaster
77,"{'_step': 234400, 'reward_f1': 0.5924308588064...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",p1gfgr7u,MsPacman


### Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown 2dgjs3c6
Seaquest 1hndgvuz
Qbert 1h0t64dp
Pong 246wv6gf
RoadRunner 35k3n89v
PrivateEye yybwqakt
Krull 31ktobwm
Kangaroo z7un794r
KungFuMaster 1vg8yjnp
MsPacman p1gfgr7u
Jamesbond 4ushilzk
Frostbite 3u3osku0
Gopher 1rzyqidw
Hero nonskx95
CrazyClimber 3w1wq49g
Freeway 22m8imw2
DemonAttack 3n0ts35g
ChopperCommand 1eyiz23m
BankHeist 197tid2b
BattleZone 354qqb55
Breakout b5i2o4qv
Boxing 3g1rxnht
Alien 3dycl85a
Asterix lgi6gcks
Amidar 1m20nhb9
Assault 3txfymh8


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### Generate json file

In [17]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)